# DataSet Performance

This notebook shows the trade-off between inserting data into a database row-by-row and as binary blobs. Inserting the data row-by-row means that we have direct access to all the data and may perform queries directly on the values of the data. On the other hand, as we shall see, this is much slower than inserting the data directly as binary blobs.

First, we choose a new location for the database to ensure that we don't add a bunch of benchmarking data to the default one.

In [1]:
import os
cwd = os.getcwd()
import qcodes as qc
qc.config["core"]["db_location"] = os.path.join(cwd, 'testing.db')


Logging hadn't been started.
Activating auto-logging. Current session state plus future input saved.
Filename       : /home/runner/.qcodes/logs/command_history.log
Mode           : append
Output logging : True
Raw input log  : False
Timestamping   : True
State          : active


Qcodes Logfile : /home/runner/.qcodes/logs/210511-4535-qcodes.log


In [2]:
%matplotlib inline
import time
import matplotlib.pyplot as plt
import numpy as np

import qcodes as qc
from qcodes.instrument.parameter import ManualParameter
from qcodes.dataset.experiment_container import (Experiment,
                                                 load_last_experiment,
                                                 new_experiment)
from qcodes.dataset.sqlite.database import initialise_database
from qcodes import load_or_create_experiment
from qcodes.dataset.measurements import Measurement

In [3]:
initialise_database()
exp = load_or_create_experiment(experiment_name='tutorial_exp', sample_name="no sample")

0it [00:00, ?it/s]

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]

Upgrading database; v0 -> v1: : 0it [00:00, ?it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

Upgrading database; v1 -> v2:   0%|          | 0/1 [00:00<?, ?it/s]

...

Upgrading database; v7 -> v8:   0%|          | 0/1 [00:00<?, ?it/s]

Upgrading database; v7 -> v8: 100%|██████████| 1/1 [00:00<00:00, 810.96it/s]


  0%|          | 0/1 [00:00<?, ?it/s]

Upgrading database; v8 -> v9:   0%|          | 0/1 [00:00<?, ?it/s]

Upgrading database; v8 -> v9: 100%|██████████| 1/1 [00:00<00:00, 491.77it/s]

Here, we define a simple function to benchmark the time it takes to insert n points with either numeric or array data type.
We will compare both the time used to call ``add_result`` and the time used for the full measurement.

In [4]:
def insert_data(paramtype, npoints, nreps=1):

    meas = Measurement(exp=exp)

    x1 = ManualParameter('x1')
    x2 = ManualParameter('x2')
    x3 = ManualParameter('x3')
    y1 = ManualParameter('y1')
    y2 = ManualParameter('y2')

    meas.register_parameter(x1, paramtype=paramtype)
    meas.register_parameter(x2, paramtype=paramtype)
    meas.register_parameter(x3, paramtype=paramtype)
    meas.register_parameter(y1, setpoints=[x1, x2, x3],
                            paramtype=paramtype)
    meas.register_parameter(y2, setpoints=[x1, x2, x3],
                            paramtype=paramtype)
    start = time.perf_counter()
    with meas.run() as datasaver:
        start_adding = time.perf_counter()
        for i in range(nreps):
            datasaver.add_result((x1, np.random.rand(npoints)),
                                 (x2, np.random.rand(npoints)),
                                 (x3, np.random.rand(npoints)),
                                 (y1, np.random.rand(npoints)),
                                 (y2, np.random.rand(npoints)))
        stop_adding = time.perf_counter()
        run_id = datasaver.run_id
    stop = time.perf_counter()
    tot_time = stop - start
    add_time = stop_adding - start_adding
    return tot_time, add_time, run_id

## Comparison between numeric/array data and binary blob

### Case1: Short experiment time

In [5]:
sizes = [1,500,1000,2000,3000,4000,5000]
t_numeric = []
t_numeric_add = []
t_array = []
t_array_add = []
for size in sizes:
    tn, tna, run_id_n =  insert_data('numeric', size)
    t_numeric.append(tn)
    t_numeric_add.append(tna)

    ta, taa, run_id_a =  insert_data('array', size)
    t_array.append(ta)
    t_array_add.append(taa)

Starting experimental run with id: 1. 
Starting experimental run with id: 2. 
Starting experimental run with id: 3. 


Starting experimental run with id: 4. 


Starting experimental run with id: 5. 


Starting experimental run with id: 6. 
Starting experimental run with id: 7. 


Starting experimental run with id: 8. 


Starting experimental run with id: 9. 


Starting experimental run with id: 10. 
Starting experimental run with id: 11. 


Starting experimental run with id: 12. 
Starting experimental run with id: 13. 


Starting experimental run with id: 14. 


In [6]:
fig, ax = plt.subplots(1,1)
ax.plot(sizes, t_numeric, 'o-', label='Inserting row-by-row')
ax.plot(sizes, t_numeric_add, 'o-', label='Inserting row-by-row: add_result only')
ax.plot(sizes, t_array, 'd-', label='Inserting as binary blob')
ax.plot(sizes, t_array_add, 'd-', label='Inserting as binary blob: add_result only')
ax.legend()
ax.set_xlabel('Array length')
ax.set_ylabel('Time (s)')
fig.tight_layout()

As shown in the latter figure, the time to setup and and close the experiment is approximately 0.4 sec. In case of small array sizes, the difference between inserting values of data as arrays and inserting them row-by-row is relatively unimportant. At larger array sizes, i.e. above 10000 points, the cost of writing data as individual datapoints starts to become important.


### Case2: Long experiment time 

In [7]:
sizes = [1,500,1000,2000,3000,4000,5000]
nreps = 100
t_numeric = []
t_numeric_add = []
t_numeric_run_ids = []
t_array = []
t_array_add = []
t_array_run_ids = []
for size in sizes:
    tn, tna, run_id_n =  insert_data('numeric', size, nreps=nreps)
    t_numeric.append(tn)
    t_numeric_add.append(tna)
    t_numeric_run_ids.append(run_id_n)

    ta, taa, run_id_a =  insert_data('array', size, nreps=nreps)
    t_array.append(ta)
    t_array_add.append(taa)
    t_array_run_ids.append(run_id_a)

Starting experimental run with id: 15. 
Starting experimental run with id: 16. 


Starting experimental run with id: 17. 


Starting experimental run with id: 18. 


Starting experimental run with id: 19. 


Starting experimental run with id: 20. 


...

Starting experimental run with id: 24. 


Starting experimental run with id: 25. 


Starting experimental run with id: 26. 


Starting experimental run with id: 27. 


Starting experimental run with id: 28. 


In [8]:
fig, ax = plt.subplots(1,1)
ax.plot(sizes, t_numeric, 'o-', label='Inserting row-by-row')
ax.plot(sizes, t_numeric_add, 'o-', label='Inserting row-by-row: add_result only')
ax.plot(sizes, t_array, 'd-', label='Inserting as binary blob')
ax.plot(sizes, t_array_add, 'd-', label='Inserting as binary blob: add_result only')
ax.legend()
ax.set_xlabel('Array length')
ax.set_ylabel('Time (s)')
fig.tight_layout()

However, as we increase the length of the experiment, as seen here by repeating the insertion 100 times, we see a big difference between inserting values of the data row-by-row and inserting it as a binary blob.

## Loading the data 

In [9]:
from qcodes.dataset.data_set import load_by_id

As usual you can load the data by using the ``load_by_id`` function but you will notice that the different storage methods
are reflected in shape of the data as it is retrieved. 

In [10]:
run_id_n = t_numeric_run_ids[0]
run_id_a = t_array_run_ids[0]

In [11]:
ds = load_by_id(run_id_n)
ds.get_parameter_data('x1')

{'x1': {'x1': array([0.68614647, 0.68614647, 0.94206258, 0.94206258, 0.89044126,
         0.89044126, 0.83238063, 0.83238063, 0.99496869, 0.99496869,
         0.40793293, 0.40793293, 0.75352762, 0.75352762, 0.45049996,
         0.45049996, 0.82025272, 0.82025272, 0.36601408, 0.36601408,
         0.10624676, 0.10624676, 0.889878  , 0.889878  , 0.10787001,
         0.10787001, 0.8919637 , 0.8919637 , 0.76057166, 0.76057166,
         0.95289299, 0.95289299, 0.89992791, 0.89992791, 0.28160383,
         0.28160383, 0.55695578, 0.55695578, 0.21075012, 0.21075012,
         0.80378756, 0.80378756, 0.45830809, 0.45830809, 0.80842013,
         0.80842013, 0.46130527, 0.46130527, 0.86102588, 0.86102588,
         0.4636359 , 0.4636359 , 0.24379365, 0.24379365, 0.62539148,
         0.62539148, 0.55035055, 0.55035055, 0.57006247, 0.57006247,
         0.34869615, 0.34869615, 0.24561403, 0.24561403, 0.43232328,
         0.43232328, 0.67638137, 0.67638137, 0.63721465, 0.63721465,
         0.76297815, 0

And a dataset stored as binary arrays

In [12]:
ds = load_by_id(run_id_a)
ds.get_parameter_data('x1')

{'x1': {'x1': array([[0.60206431],
         [0.60206431],
         [0.17186896],
         [0.17186896],
         [0.94608233],
         [0.94608233],
         [0.50841869],
         [0.50841869],
         [0.07968909],
         [0.07968909],
         [0.79907836],
         [0.79907836],
         [0.65878987],
         [0.65878987],
         [0.76035486],
         [0.76035486],
         [0.39897954],
         [0.39897954],
         [0.60012671],
         [0.60012671],
         [0.08377863],
         [0.08377863],
         [0.62672297],
         [0.62672297],
         [0.07094088],
         [0.07094088],
         [0.1628591 ],
         [0.1628591 ],
         [0.1887213 ],
         [0.1887213 ],
         [0.57449834],
         [0.57449834],
         [0.20143148],
         [0.20143148],
         [0.46225397],
         [0.46225397],
         [0.86310608],
         [0.86310608],
         [0.31916056],
         [0.31916056],
         [0.18550274],
         [0.18550274],
         [0.78893887],